# End-to-End Demo

## Setup

In [1]:
%pip install dataconfigs

In [2]:
from dataclasses import dataclass, is_dataclass, InitVar, field
from dataconfigs import configurable, show_config_params

## Demo

### Config

In [3]:
@dataclass
class MyConfig:
    """My config class

    Args:
        param1: Parameter 1 (required). Defaults to 0.
        param2: Parameter 2 (optional). Defaults to param2.
        param3: Parameter 3 (not configurable and not visible).
    """
    param1: InitVar[int]
    param2: str = "param2"
    param3: float = field(init=False)

    def __post_init__(self, param1: int) -> None:
        # We set the param 3 here
        self.param3 = param1 / 3

### Configurable

In [4]:
@configurable
class MyConfigurable(MyConfig):
    def __init__(self, attr1=1, attr2=2):
        # This is non-config attr
        self.attr = attr1 + attr2
        print(f"{self.attr=}")
        
        # Formerly, config attrs
        print(f"{self.param2=}") 
        print(f"{self.param3=}")
        assert not hasattr(self, "param1") # It was InitVar only!

# Even though MyConfig is a dataclass, MyConfigurable DOES NOT inherit
# from it, i.e., only attributes are copied. Very convenient because,
# semantically, PyLint still links Configurable params to MyConfig
assert not is_dataclass(MyConfigurable)

### Instance

In [5]:
# Init configurable (note how config param2 is overridden)
obj = MyConfigurable(attr2=99, param1=1, param2="PARAM2")

self.attr=100
self.param2='PARAM2'
self.param3=0.3333333333333333


In [6]:
show_config_params(MyConfigurable)
show_config_params(obj)

MyConfigurable class config parameters (global defaults):
        * param2 (str): Parameter 2 (optional). Defaults to param2.
        * param1 (int): Parameter 1 (required). Defaults to 0.
MyConfigurable instance config parameters (local defaults):
        * param2 (str): Parameter 2 (optional). Defaults to PARAM2.
        * param1 (int): Parameter 1 (required). Defaults to 1.


### Alternative 1: Outer Config

In [7]:
@configurable(config=MyConfig) # <- Pass MyConfig as `config` parameter
class MyConfigurable:
    def __init__(self, attr1=1, attr2=2):
        pass

# This is equivalent to the previous example
show_config_params(MyConfigurable)

MyConfigurable class config parameters (global defaults):
        * param2 (str): Parameter 2 (optional). Defaults to param2.
        * param1 (int): Parameter 1 (required). Defaults to 0.


### Alternative 2: Inner Config

In [8]:
@configurable
class MyConfigurable:
    ############### Any Inner configs can be defined here ##############
    @dataclass
    class MyConfig:
        """
        Args:
            param1: Parameter 1 (required). Defaults to 0.
            param2: Parameter 2 (optional). Defaults to param2.
        """
        param1: InitVar[int]
        param2: str = "param2"
        param3: float = field(init=False)

        def __post_init__(self, param1: int) -> None:
            # We set the param 3 here
            self.param3 = param1 / 3

    ######################## Rest of the class #########################
    def __init__(self, attr1=1, attr2=2):
        pass

# This is equivalent to the previous example
show_config_params(MyConfigurable)

MyConfigurable class config parameters (global defaults):
        * param2 (str): Parameter 2 (optional). Defaults to param2.
        * param1 (int): Parameter 1 (required). Defaults to 0.


### Multiple Configs

In [9]:
@dataclass
class MyConfig2:
    """
    :param param4: The fourth parameter from another config.
    """
    param4: bool = False

@configurable(config=(MyConfig, MyConfig2)) # <- pass here, extend, or define as inner
class MyConfigurable:
    def __init__(self, attr1=1, attr2=2):
        print(f"{self.param2=}, {self.param3=}, {self.param4=}") 

# param4 is now shown as configurable parameter as well
# Note how "Defaults to" is also added automatically
show_config_params(MyConfigurable(param1=1))

self.param2='param2', self.param3=0.3333333333333333, self.param4=False
MyConfigurable instance config parameters (local defaults):
        * param2 (str): Parameter 2 (optional). Defaults to param2.
        * param1 (int): Parameter 1 (required). Defaults to 1.
        * param4 (bool): The fourth parameter from another config. Defaults to False.
